In [ ]:
##this 

In [ ]:
####run the jupyter notebook for the tomography model 
import numpy as np
import matplotlib.pyplot as plt
import os
PROJECT='/data2/yjgao/data/DATASET/package_forthesis/'

In [ ]:
def launch_forward(dir_run,dir_event,simtype,modelfile,dir_storesyn):
    os.chdir(dir_run)
    os.chdir(dir_event)
    from shutil import copyfile
    from distutils.dir_util import copy_tree
    copyfile(modelfile,'DATA/profile.xyz')
    copyfile('../run_this_example.sh','run_this_example.sh')
    os.system('ln -s specfem2d-devel/utils/change_simulation_type.pl')
    os.system('perl change_simulation_type.pl -f') 
    print('start forward modelling',dir_event)
    os.system('bash run_this_example.sh')
    print('finished forward modelling')
    ###store the synthetics to the dir observed or syn for backup
    copy_tree('OUTPUT_FILES',dir_storesyn)
    os.chdir(dir_run)
    #print(pwd)

In [ ]:
####compare smoothed model with 1D model 
modelfile=PROJECT+'MODELS/profile_obs.xyz'
from datetime import datetime
start=datetime.now()
launch_forward(PROJECT,'EVENT2','forward',modelfile,'REF_SEM')
print (datetime.now()-start)

modelfile=PROJECT+'MODELS/profile_obs_smooth.xyz'
from datetime import datetime
start=datetime.now()
launch_forward(PROJECT,'EVENT2','forward',modelfile,'REF_SEM_smooth')
print (datetime.now()-start)

In [ ]:
def read_source(dir_source):
    file1 = open(dir_source, 'r')
    Lines = file1.readlines()
    x=np.float(Lines[2].split('=')[1].split('#')[0])
    z=np.float(Lines[3].split('=')[1].split('#')[0])
    return x,z
def read_receiver(dir_receiver):
    import re
    file1 = open(dir_receiver, 'r')
    Lines = file1.readlines()
    number=len(Lines)
    receiver_station=[]
    receiver_net=[]
    receiver_x=np.zeros(number)
    receiver_z=np.zeros(number)
    for _i, line in enumerate(Lines):
        station,network,x,z,nun,nun=re.findall(r'\S+', line)
        #print(station)
        receiver_station.append(station)
        receiver_net.append(network)
        receiver_x[_i]=np.float(x)
        receiver_z[_i]=np.float(z)
    return receiver_station,receiver_net,receiver_x,receiver_z

def launch_adjoint(dir_run,dir_event,dir_storesyn,modelfile):
    from datetime import datetime
    os.chdir(dir_run)
    os.chdir(dir_event)
    from shutil import copyfile
    import shutil
    from distutils.dir_util import copy_tree
    #copyfile(modelfile,'DATA/profile.xyz')
    #os.remove('change_simulation_type.pl')
    #s.system('ln -s /home/yjgao/install/specfem2d-devel/utils/change_simulation_type.pl')
    #s.system('perl change_simulation_type.pl -f') 
    copyfile(modelfile,'DATA/profile.xyz')
    copyfile('../run_this_example_kernel.sh','run_this_example_kernel.sh')
    print('start adjoint modelling',dir_event)
    os.system('bash run_this_example_kernel.sh')
    print('finished adjoint modelling')
    ###store the synthetics to the dir observed or syn for backup
    #!cat OUTPUT_FILES/proc000*_rho_kappa_mu_kernel.dat > OUTPUT_FILES/rho_kappa_mu_kernel.dat
    ####prepared to 
    os.system('cat OUTPUT_FILES/proc000*_rhop_alpha_beta_kernel.dat > OUTPUT_FILES/rho_alpha_beta_kernel.dat')
    shutil.copy('OUTPUT_FILES/rho_alpha_beta_kernel.dat', dir_storesyn)
    os.chdir(dir_run)
def adj_calculate(dir_run,dir_event,dir_obs,dir_syn,adj_src_type,stationpath):
    import obspy
    import pyadjoint
    import numpy as np

    from obspy.taup import TauPyModel
    from obspy.core.trace import Trace
    ##guarantee to change back to the project directory
    os.chdir(dir_run)
    os.chdir(dir_event)
    ##### read source locations
    source_x,source_depth_in_m=read_source('DATA/SOURCE')
    ##### read receiver locations
    receiver_station,receiver_net,receiver_x,receiver_z=read_receiver(stationpath)
    import toml
    Misfits_all=0.0
    fh = open(dir_syn+'misfit.txt','w')
    for i, station_name in enumerate(receiver_station):
        #print(i)
        #print(np.int(i))
        distance=np.abs(receiver_x[i]-source_x)/111.0/1000.0
        model = TauPyModel(model="iasp91")
        # adapted 
        arrivals = model.get_travel_times(source_depth_in_km=-source_depth_in_m/1000,distance_in_degree=distance) 
        time=arrivals[0].time
        if time > 400-15:
            window=[]
        else:
            window=[[time-5,time+25]]
        print(window)
        network=receiver_net[i]
        station=receiver_station[i]
        data_hetero=np.loadtxt(dir_syn+network+'.'+station+'.BXZ'+'.semd')
        data_hetero_new=data_hetero.swapaxes(0,1)
    #from obspy.core.trace import Trace
        tr=Trace(data=data_hetero_new[1])
        tr.stats.delta=0.02  ###here should be adapted to the DATA/Par_file
        tr.stats.network = network
        tr.stats.station = station
        tr.stats.channel = 'BXZ'
        data_hetero_obs=np.loadtxt(dir_obs+network+'.'+station+'.BXZ'+'.semd')                           
        data_hetero_new_obs=data_hetero_obs.swapaxes(0,1)
        tr_obs=Trace(data=data_hetero_new_obs[1])
        tr_obs.stats.delta=0.02
        tr_obs.stats.network = network
        tr_obs.stats.station = station
        tr_obs.stats.channel = 'BXZ'
        ###here input a minum and maximum period
        configure=pyadjoint.config.Config(10,120,measure_type='dt')
        print(configure)
        adj=pyadjoint.calculate_adjoint_source(
        adj_src_type=adj_src_type, observed=tr_obs, synthetic=tr, min_period=5, max_period=100,
        config=configure, window=window, plot=True);
        Misfits_all+=adj.misfit
        adj.write(filename='SEM/'+network+'.'+station+'.BXZ'+'.adj',format="SPECFEM", time_offset=-12)
        
        adj=pyadjoint.calculate_adjoint_source(
        adj_src_type=adj_src_type, observed=tr_obs, synthetic=tr, min_period=5, max_period=100,
        config=configure, window=[], plot=True);
        adj.write(filename='SEM/'+network+'.'+station+'.BXY'+'.adj',format="SPECFEM", time_offset=-12)
        
        data_hetero=np.loadtxt(dir_syn+network+'.'+station+'.BXX'+'.semd')
        data_hetero_new=data_hetero.swapaxes(0,1)
        tr=Trace(data=data_hetero_new[1])
        tr.stats.delta=0.02
        tr.stats.network = network
        tr.stats.station = station
        tr.stats.channel = 'BXX'
        data_hetero_obs=np.loadtxt(dir_obs+network+'.'+station+'.BXX' +'.semd')                           
        data_hetero_new_obs=data_hetero_obs.swapaxes(0,1)
        tr_obs=Trace(data=data_hetero_new_obs[1])
        tr_obs.stats.delta=0.02
        tr_obs.stats.network = network
        tr_obs.stats.station = station
        tr_obs.stats.channel = 'BXX'
        adj=pyadjoint.calculate_adjoint_source(
        adj_src_type=adj_src_type, observed=tr_obs, synthetic=tr, min_period=5, max_period=100,
        config=configure, window=[], plot=True);
        print(adj)
        adj.write(filename='SEM/'+network+'.'+station+'.BXX'+'.adj',format="SPECFEM", time_offset=-12)
    fh.write(dir_syn+' ' + dir_event +' ' + str(Misfits_all)+"\n")
    fh.close()
    os.chdir(dir_run)

In [ ]:
print('ccc')
list_events=['EVENT2']
for _i, event in enumerate(list_events):
    adj_calculate(PROJECT,event,'REF_SEM/','REF_SEM_smooth/','ccc','DATA/STATIONS')
list_events=['EVENT2']
for _i, event in enumerate(list_events):
    launch_adjoint(PROJECT,event,'REF_SEM_smooth/',modelfile)
!cp EVENT2/REF_SEM_smooth/rho_alpha_beta_kernel.dat rho_alpha_beta_kernel_ccc_full2.dat 
!mkdir adj_ccc
!cp EVENT1/SEM/*.BXZ.adj adj_ccc/


In [ ]:
list_events=['EVENT2']
modelfile=PROJECT+'MODELS/profile_obs_smooth.xyz'
print('tf')
for _i, event in enumerate(list_events):
    adj_calculate(PROJECT,event,'REF_SEM/','REF_SEM_smooth/','tf_phase_misfit','DATA/STATIONS')
list_events=['EVENT2']
for _i, event in enumerate(list_events):
    launch_adjoint(PROJECT,event,'REF_SEM_smooth/',modelfile)
!cp EVENT1/REF_SEM_smooth/rho_alpha_beta_kernel.dat rho_alpha_beta_kernel_tf_full2.dat 
!mkdir adj_tf
!cp EVENT1/SEM/*.BXZ.adj adj_tf/

###########################################################
print('ccc')
list_events=['EVENT2']
for _i, event in enumerate(list_events):
    adj_calculate(PROJECT,event,'REF_SEM/','REF_SEM_smooth/','ccc','DATA/STATIONS')
list_events=['EVENT2']
for _i, event in enumerate(list_events):
    launch_adjoint(PROJECT,event,'REF_SEM_smooth/',modelfile)
!cp EVENT1/REF_SEM_smooth/rho_alpha_beta_kernel.dat rho_alpha_beta_kernel_ccc_full2.dat 
!mkdir adj_ccc
!cp EVENT1/SEM/*.BXZ.adj adj_ccc/

###########################################################

print('cc')
list_events=['EVENT2']
for _i, event in enumerate(list_events):
    adj_calculate(PROJECT,event,'REF_SEM/','REF_SEM_smooth/','cc_traveltime_misfit','DATA/STATIONS')
list_events=['EVENT2']
for _i, event in enumerate(list_events):
    launch_adjoint(PROJECT,event,'REF_SEM_smooth/',modelfile)
!cp EVENT1/REF_SEM_smooth/rho_alpha_beta_kernel.dat rho_alpha_beta_kernel_cc_full2.dat 
!mkdir adj_cc
!cp EVENT1/SEM/*.BXZ.adj adj_cc/


###########################################################
print('mt')
list_events=['EVENT2']
for _i, event in enumerate(list_events):
    adj_calculate(PROJECT,event,'REF_SEM/','REF_SEM_smooth/','multitaper_misfit','DATA/STATIONS')
list_events=['EVENT2']
for _i, event in enumerate(list_events):
    launch_adjoint(PROJECT,event,'REF_SEM_smooth/',modelfile)
!cp EVENT1/REF_SEM_smooth/rho_alpha_beta_kernel.dat rho_alpha_beta_kernel_mt_full2.dat 
!mkdir adj_mt
!cp EVENT1/SEM/*.BXZ.adj adj_mt/

############################################################
print('wm')
list_events=['EVENT2']
for _i, event in enumerate(list_events):
    adj_calculate(PROJECT,event,'REF_SEM/','REF_SEM_smooth/','waveform_misfit','DATA/STATIONS')
list_events=['EVENT2']
for _i, event in enumerate(list_events):
    launch_adjoint(PROJECT,event,'REF_SEM_smooth/',modelfile)
!cp EVENT1/REF_SEM_smooth/rho_alpha_beta_kernel.dat rho_alpha_beta_kernel_wm_full2.dat 
!mkdir adj_wm
!cp EVENT1/SEM/*.BXZ.adj adj_wm/


In [ ]:

import os.path
import sys
import numpy as np

try:
    import matplotlib.pyplot as plt
except:
    print("Error importing pyplot from matplotlib, please install matplotlib package first...")
    sys.tracebacklimit=0
    raise Exception("Importing matplotlib failed")
###controlling the speed for smoothing!
def grid(x, y, z, resX=500, resY=500):
    """
    Converts 3 column data to matplotlib grid
    """
    from scipy.interpolate import griddata
    #from scipy.interpolate import griddata

    xi = np.linspace(min(x), max(x), resX)
    yi = np.linspace(min(y), max(y), resY)

    # mlab version
    Z = griddata((x, y), z, (xi[None,:], yi[:,None]), method='linear')
    # scipy version
    #Z = griddata((x, y), z, (xi[None,:], yi[:,None]), method='cubic')

    X, Y = np.meshgrid(xi, yi)
    return X, Y, Z

def plot_kernels(filename,show=False,EVENT='EVENT',total_max=1e-10,figurename='test.png'):
    """
    plots ASCII kernel file
    """
    #print "plotting kernel file: ",filename
    #print ""

    data = np.loadtxt(filename)

    # checks data
    if data.ndim != 2:
        #print "Error: wrong data dimension for kernel file",data.ndim
        sys.tracebacklimit=0
        raise Exception("Invalid data dimension")

    # checks array
    if len(data[1,:]) != 5:
        #print "data shape  : ",data.shape
        #print "data lengths: ",len(data[:,1]),len(data[1,:])
        #print "Error: wrong data format for kernel file",data.shape
        sys.tracebacklimit=0
        raise Exception("Invalid data format")

    # splits up data
    x = data[:,0]
    y = data[:,1]

    #print "dimensions:"
    #print "  x-range min/max = %f / %f" % (x.min(), x.max())
    #print "  y-range min/max = %f / %f" % (y.min(), y.max())
    #print ""

    z1 = data[:,2] # e.g. rho
    z2 = data[:,3] # e.g. alpha
    z3 = data[:,4] # e.g. beta

    # names like
    #   rhop_alpha_beta_kernel.dat
    # or
    #   proc000000_rhop_alpha_beta_kernel.dat
    name = os.path.basename(filename)

    name_kernels = str.split(name,"_")
    if len(name_kernels) == 4:
        kernel1 = 'K_' + name_kernels[0] # rhop
        kernel2 = 'K_' + name_kernels[1] # alpha
        kernel3 = 'K_' + name_kernels[2] # beta
    elif len(name_kernels) == 5:
        kernel1 = 'K_' + name_kernels[1]
        kernel2 = 'K_' + name_kernels[2]
        kernel3 = 'K_' + name_kernels[3]
    else:
        kernel1 = 'K_1'
        kernel2 = 'K_2'
        kernel3 = 'K_3'

    #print "statistics:"
    #print "  %12s : min/max = %e / %e" % (kernel1,z1.min(),z1.max())
    #print "  %12s : min/max = %e / %e" % (kernel2,z2.min(),z2.max())
    #print "  %12s : min/max = %e / %e" % (kernel3,z3.min(),z3.max())
    #print ""

    #total_max = 0.1*abs(z2).max()
    print(total_max)
    #print "  data max = ",total_max
    #print ""
    #print(total_max)
    #total_max = 1.e-8

    # setup figure (with 3 subplots)
    fig, axes = plt.subplots(nrows=3, ncols=1,figsize=(10,10))

    for i,ax in enumerate(axes.flat,start=1):
        # top
        if i == 1:
            X, Y, Z = grid(x,y,z1)
            ax.set_title("Kernels")
            ax.set_ylabel('RHO')
        elif i == 2:
            X, Y, Z = grid(x,y,z2)
            ax.set_ylabel('VP')
        elif i == 3:
            X, Y, Z = grid(x,y,z3)
            ax.set_ylabel('VS')

        #colormap = 'jet'
        colormap = 'RdBu'
        print(np.max(np.abs(Z)))
        im = ax.imshow(Z/np.max(np.abs(Z)),vmax=0.2, vmin=-0.2,extent=[0,45*111,0,1000],cmap=colormap)
               # im = ax.imshow(Z,vmax=total_max, vmin=total_min,extent=[0,45*111,0,1000],cmap=colormap.reversed())
        ax.plot([18*111,25*111,25*111,18*111,18*111],[500,500,660,660,500],'black')
        ax.plot([0,45*111],[410,410],'black')
        ax.plot([0,45*111],[660,660],'black')
        ax.text(10,390,'410 km', fontsize=9)
        ax.text(10,640,'660 km', fontsize=9)
        ax.invert_yaxis()
        #ax.set_xlim(2000000,3000000)
        #ax.set_ylim(-200000,0)
    # moves plots together
    fig.subplots_adjust(hspace=0)
    #plt.setp([a.get_xticklabels() for a in fig.axes[:-1]], visible=False)
    
    # colorbar
    fig.colorbar(im, ax=axes.ravel().tolist())
    #plt.xlim(2500000,3000000)
    #plt.ylim(-200000,0)
    #fig.colorbar(im, ax=axes.ravel().tolist(),orientation='horizontal')

    # show the figure
    if show:
        plt.show()

    # saves kernel figure as file
    dir = os.path.dirname('.')
    name_without_ending = str.split(name,".")[0]
    outfile = dir + "/" + name_without_ending + ".png"
    fig.savefig(figurename, format="png",  dpi=300)

    #print "*****"
    #print "plotted file: ",outfile
    #print "*****"
    #print ""

In [ ]:
def plot_kernel_different(kernellist,misfitname):
    fig, axes = plt.subplots(nrows=5, ncols=1,figsize=(15,18))
    for _i,filename in enumerate(kernellist):
        print(_i)
        #ax.subplot()
        data = np.loadtxt(filename)

    # checks data
        if data.ndim != 2:
        #print "Error: wrong data dimension for kernel file",data.ndim
            sys.tracebacklimit=0
            raise Exception("Invalid data dimension")

    # checks array
        if len(data[1,:]) != 5:
        #print "data shape  : ",data.shape
        #print "data lengths: ",len(data[:,1]),len(data[1,:])
        #print "Error: wrong data format for kernel file",data.shape
            sys.tracebacklimit=0
            raise Exception("Invalid data format")

        x = data[:,0]
        y = data[:,1]
      #  z1 = data[:,2] # e.g. rho
        z2 = data[:,3] # e.g. alpha
       # z3 = data[:,4] # e.g. beta

        X, Y, Z = grid(x,y,z2)
        colormap = 'RdBu'
        print(np.max(np.abs(Z)))
        im = axes[_i].imshow(Z/np.max(np.abs(Z)),vmax=0.05, vmin=-0.05,extent=[0,45*111,0,1000],cmap=colormap)
               # im = ax.imshow(Z,vmax=total_max, vmin=total_min,extent=[0,45*111,0,1000],cmap=colormap.reversed())
        axes[_i].plot([18*111,25*111,25*111,18*111,18*111],[500,500,660,660,500],'black')
        axes[_i].plot([0,45*111],[410,410],'black')
        axes[_i].plot([0,45*111],[660,660],'black')
        #axes[_i].text(10,390,'410 km', fontsize=9)
        #axes[_i].text(10,640,'660 km', fontsize=9)
        axes[_i].invert_yaxis()
        axes[_i].text(2000,900,misfitname[_i])
        #axes[_i].set_xlim(1000,3500)
    fig.colorbar(im, ax=axes.ravel().tolist(),shrink=0.5,label='Normalised Misfit Kernel (P wave)')
    fig.savefig('kernel_compare_full.pdf')
       #axes[_i].invert_yaxis()

In [ ]:
kernellist=['rho_alpha_beta_kernel_cc_full.dat','rho_alpha_beta_kernel_mt_full.dat','rho_alpha_beta_kernel_wm_full.dat',\
           'rho_alpha_beta_kernel_tf_full.dat','rho_alpha_beta_kernel_ccc_full.dat']
misfitname=['CC time shift','multitaper time shift','waveform L2',\
           'TF phase shift','CCC misfit']
plot_kernel_different(kernellist,misfitname)

In [ ]:
####compare smoothed model with 1D model 
modelfile=PROJECT+'MODELS/profile_obs_smooth.xyz'
from datetime import datetime
start=datetime.now()
launch_forward(PROJECT,'EVENT2','forward',modelfile,'REF_SEM_smooth')
print (datetime.now()-start)
modelfile=PROJECT+'MODELS/profile_syn.xyz'
from datetime import datetime
start=datetime.now()
launch_forward(PROJECT,'EVENT2','forward',modelfile,'REF_SEM_SYN')

In [ ]:
list_events=['EVENT2']
print('tf')
modelfile=PROJECT+'MODELS/profile_syn.xyz'
for _i, event in enumerate(list_events):
    adj_calculate(PROJECT,event,'REF_SEM_smooth/','REF_SEM_SYN/','tf_phase_misfit','DATA/STATIONS')
list_events=['EVENT2']
for _i, event in enumerate(list_events):
    launch_adjoint(PROJECT,event,'REF_SEM_SYN/',modelfile)
!cp EVENT2/REF_SEM_smooth/rho_alpha_beta_kernel.dat rho_alpha_beta_kernel_tf_full_smoothsyn2.dat 
!mkdir adj_tf_smoothsyn
!cp EVENT1/SEM/*.BXZ.adj adj_tf_smoothsyn/

In [ ]:
#####adjoint simulations
list_events=['EVENT2']
print('tf')
for _i, event in enumerate(list_events):
    adj_calculate(PROJECT,event,'REF_SEM_smooth/','REF_SEM_SYN/','tf_phase_misfit','DATA/STATIONS')
list_events=['EVENT2']
for _i, event in enumerate(list_events):
    launch_adjoint(PROJECT,event,'REF_SEM_SYN/',modelfile)
!cp EVENT2/REF_SEM_smooth/rho_alpha_beta_kernel.dat rho_alpha_beta_kernel_tf_full_smoothsyn2.dat 
!mkdir adj_tf_smoothsyn
!cp EVENT2/SEM/*.BXZ.adj adj_tf_smoothsyn/

###########################################################
print('ccc')
list_events=['EVENT2']
for _i, event in enumerate(list_events):
    adj_calculate(PROJECT,event,'REF_SEM_smooth/','REF_SEM_SYN/','ccc','DATA/STATIONS')
list_events=['EVENT2']
for _i, event in enumerate(list_events):
    launch_adjoint(PROJECT,event,'REF_SEM_SYN/',modelfile)
!cp EVENT2/REF_SEM_smooth/rho_alpha_beta_kernel.dat rho_alpha_beta_kernel_ccc_full_smoothsyn2.dat 
!mkdir adj_ccc_smoothsyn
!cp EVENT1/SEM/*.BXZ.adj adj_ccc_smoothsyn/

###########################################################

print('cc')
list_events=['EVENT2']
for _i, event in enumerate(list_events):
    adj_calculate(PROJECT,event,'REF_SEM_smooth/','REF_SEM_SYN/','cc_traveltime_misfit','DATA/STATIONS')
list_events=['EVENT2']
for _i, event in enumerate(list_events):
    launch_adjoint(PROJECT,event,'REF_SEM_SYN/',modelfile)
!cp EVENT2/REF_SEM_smooth/rho_alpha_beta_kernel.dat rho_alpha_beta_kernel_cc_full_smoothsyn2.dat 
!mkdir adj_cc_smoothsyn
!cp EVENT1/SEM/*.BXZ.adj adj_cc_smoothsyn/


###########################################################
print('mt')
list_events=['EVENT2']
for _i, event in enumerate(list_events):
    adj_calculate(PROJECT,event,'REF_SEM_smooth/','REF_SEM_SYN/','multitaper_misfit','DATA/STATIONS')
list_events=['EVENT2']
for _i, event in enumerate(list_events):
    launch_adjoint(PROJECT,event,'REF_SEM_SYN/',modelfile)
!cp EVENT2/REF_SEM_smooth/rho_alpha_beta_kernel.dat rho_alpha_beta_kernel_mt_full_smoothsyn2.dat 
!mkdir adj_mt_smoothsyn
!cp EVENT1/SEM/*.BXZ.adj adj_mt_smoothsyn/

############################################################
print('wm')
list_events=['EVENT2']
for _i, event in enumerate(list_events):
    adj_calculate(PROJECT,event,'REF_SEM_smooth/','REF_SEM_SYN/','waveform_misfit','DATA/STATIONS')
list_events=['EVENT2']
for _i, event in enumerate(list_events):
    launch_adjoint(PROJECT,event,'REF_SEM_SYN/',modelfile)
!cp EVENT2/REF_SEM_smooth/rho_alpha_beta_kernel.dat rho_alpha_beta_kernel_wm_full_smoothsyn2.dat 
!mkdir adj_wm_smoothsyn
!cp EVENT1/SEM/*.BXZ.adj adj_wm_smoothsyn/

In [ ]:
def plot_kernel_different(kernellist,misfitname):
    fig, axes = plt.subplots(nrows=5, ncols=1,figsize=(15,18))
    for _i,filename in enumerate(kernellist):
        print(_i)
        #ax.subplot()
        data = np.loadtxt(filename)

    # checks data
        if data.ndim != 2:
        #print "Error: wrong data dimension for kernel file",data.ndim
            sys.tracebacklimit=0
            raise Exception("Invalid data dimension")

    # checks array
        if len(data[1,:]) != 5:
        #print "data shape  : ",data.shape
        #print "data lengths: ",len(data[:,1]),len(data[1,:])
        #print "Error: wrong data format for kernel file",data.shape
            sys.tracebacklimit=0
            raise Exception("Invalid data format")

        x = data[:,0]
        y = data[:,1]
      #  z1 = data[:,2] # e.g. rho
        z2 = data[:,3] # e.g. alpha
       # z3 = data[:,4] # e.g. beta

        X, Y, Z = grid(x,y,z2)
        colormap = 'RdBu'
        print(np.max(np.abs(Z)))
        im = axes[_i].imshow(Z/np.max(np.abs(Z)),vmax=0.02, vmin=-0.02,extent=[0,45*111,0,1000],cmap=colormap)
               # im = ax.imshow(Z,vmax=total_max, vmin=total_min,extent=[0,45*111,0,1000],cmap=colormap.reversed())
        axes[_i].plot([18*111,25*111,25*111,18*111,18*111],[500,500,660,660,500],'black')
        axes[_i].plot([0,45*111],[410,410],'black')
        axes[_i].plot([0,45*111],[660,660],'black')
        #axes[_i].text(10,390,'410 km', fontsize=9)
        #axes[_i].text(10,640,'660 km', fontsize=9)
        axes[_i].invert_yaxis()
        axes[_i].text(2000,900,misfitname[_i])
        #axes[_i].set_xlim(1000,3500)
    fig.colorbar(im, ax=axes.ravel().tolist(),shrink=0.5,label='Normalised Misfit Kernel (P wave)')
    fig.savefig('kernel_compare_full_smooth2.pdf')
       #axes[_i].invert_yaxis()

In [ ]:
kernellist=['rho_alpha_beta_kernel_cc_full_smoothsyn2.dat','rho_alpha_beta_kernel_mt_full_smoothsyn2.dat',\
            'rho_alpha_beta_kernel_wm_full_smoothsyn2.dat',\
           'rho_alpha_beta_kernel_tf_full_smoothsyn2.dat','rho_alpha_beta_kernel_ccc_full_smoothsyn2.dat']
misfitname=['CC time shift','multitaper time shift','waveform L2',\
           'TF phase shift','CCC misfit']
plot_kernel_different(kernellist,misfitname)